In [2]:
# Following: https://towardsdatascience.com/using-word2vec-to-analyze-news-headlines-and-predict-article-success-cdeda5f14751

import pandas as pd
import gensim
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re
import xgboost as xgb
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lnajt\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [3]:
main_data = pd.read_csv('../Data/wsb_cleaned.csv')
main_data.head()

,Unnamed: 0,title,author,created_datetime_utc,selftext,url,upvote_ratio,ups,total_awards_received,num_comments,num_crossposts,is_self,is_video,media_only,id,created_utc,selftext_html,author_fullname
0,0,Good time to get on $BLUE,None,2020-01-01 23:53:34,[deleted]\n,https://www.reddit.com/r/wallstreetbets/commen...,0.75,6,0,17,0,True,False,False,eipxnr,1.577923e+09,"<!-- SC_OFF --><div class=""md""><p>[deleted]</p...",NaN
1,1,i wanna buy a call but i don't want to bet too...,None,2020-01-01 23:53:28,[removed]\n,https://www.reddit.com/r/wallstreetbets/commen...,0.45,0,0,22,0,True,False,False,eipxm4,1.577923e+09,"<!-- SC_OFF --><div class=""md""><p>[removed]</p...",NaN
2,2,Buy INTU - DD,unclerudy,2020-01-01 23:50:05,https://www.schaeffersresearch.com/content/ana...,https://www.reddit.com/r/wallstreetbets/commen...,0.70,5,0,4,0,True,False,False,eipw3g,1.577923e+09,"<!-- SC_OFF --><div class=""md""><p><a href=""htt...",t2_7uwqo
3,3,"If you thought your 2019 was bad, at least you...",bd_one,2020-01-01 23:47:09,NaN,https://i.imgflip.com/3kywdh.jpg,1.00,1,0,1,0,False,False,False,eipusd,1.577922e+09,NaN,t2_13de6v
4,4,I will remember,IpMedia,2020-01-01 23:30:47,[removed]\n,https://www.reddit.com/r/wallstreetbets/commen...,0.86,11,0,4,0,True,False,False,eipn7c,1.577921e+09,"<!-- SC_OFF --><div class=""md""><p>[removed]</p...",t2_bhcyy


In [4]:
regex = re.compile('[^a-zA-Z ]')
for col in ['title', 'author', 'selftext']:
    main_data[col] = main_data[col].apply(lambda x : regex.sub('', str(x).lower() ))

In [5]:
article_titles = main_data['title']
titles_list = [title for title in article_titles]

In [6]:
big_title_string = ' '.join(titles_list)

from nltk.tokenize import word_tokenize

In [7]:
tokens = word_tokenize(big_title_string)

In [8]:
words = [word.lower() for word in tokens if word.isalpha()]

In [9]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

words = [word for word in words if not word in stop_words]

In [10]:
words[:10]

['good', 'time', 'get', 'blue', 'wan', 'na', 'buy', 'call', 'dont', 'want']

In [13]:
text = main_data['title']

#sentences = [nltk.word_tokenize(sent) for sent in nltk.sent_tokenize(text)]
sentences = [x.split(' ') for x in text]

model = gensim.models.Word2Vec(sentences = sentences,  min_count=10, vector_size=200, epochs = 4)


In [ ]:
# Filter the list of vectors to include only those that Word2Vec has a vector for
vector_list = [model[word] for word in words if word in model.vocab]

# Create a list of the words corresponding to these vectors
words_filtered = [word for word in words if word in model.vocab]

# Zip the words together with their vector representations
word_vec_zip = zip(words_filtered, vector_list)

# Cast to a dict so we can turn it into a DataFrame
word_vec_dict = dict(word_vec_zip)
df = pd.DataFrame.from_dict(word_vec_dict, orient='index')
df.head(3)

In [ ]:
from sklearn.manifold import TSNE

# Initialize t-SNE
tsne = TSNE(n_components = 2, init = 'random', random_state = 10, perplexity = 100)

# Use only 400 rows to shorten processing time
tsne_df = tsne.fit_transform(df[:400])